In [1]:
import heapq as hq

class Passenger:
    def __init__(self, BM, exit_time):
        self.bm = BM
        self.exit = exit_time
    def __lt__(self, other):
        return (self.exit, self.bm) < \
               (other.exit, other.bm)

class BM:
    def __init__(self, passenger, capacity, stop, itinerary):
        self.passenger = passenger
        self.capacity = capacity
        self.stop = stop
        self.itinerary = itinerary
    def __le__(self, other):
        pass
    
class Event:
    def __init__(self, time, action, bm):
        self.time = time
        self.action = action
        self.bm = bm
    def __lt__(self, other):
        return (self.time, self.bm) < (other.time, other.bm)
    def __le__(self, other):
        return (self.time, self.bm) <= (other.time, other.bm)

In [3]:
#passengers = [[{'bm': 0, 'exit_time': 120}]*100 , [{'bm': 1 and 0, 'exit_time': 120}]*40]
passengers = [[{'station': 1, 'exit_time': 60}]*8 + [{'station': 2, 'exit_time':120}]*2,
              [{'station': 0, 'exit_time':110}]*8 + [{'station': 2, 'exit_time': 60}]*2,
              [{'station': 0, 'exit_time': 50}]*0 + [{'station': 2, 'exit_time':110}]*0]
bms = [{'n_passengers': 0, 'capacity': 3, 'stop': 0, 'itinerary': [0, 1], 'name': 0},
       {'n_passengers': 0, 'capacity': 5, 'stop': 1, 'itinerary': [0, 1, 2], 'name': 1}]

event0 = Event(0,'load',0)
event1 = Event(0,'load',1)
eventq = []
hq.heappush(eventq, event0)
hq.heappush(eventq, event1)

#def solve(eventq):
while eventq:
    event = hq.heappop(eventq)
    bm = bms[event.bm]
    current_node = bm['itinerary'][bm['stop'] % len(bm['itinerary'])]
    next_node = bm['itinerary'][(bm['stop']+1) % len(bm['itinerary'])]
    print('Fahrzeug {} ist am knoten {}'.format(bm['name'],current_node))
    print('Nächterhalt des Fahrzeug {} ist {}'.format(bm['name'],next_node))
    if event.action == 'load':
        print('Fahrzeug {} load'.format(bm['name']))
        free = bm['capacity'] - bm['n_passengers']
        print('Fahrzeug {} hat {} Pläze frei'.format(bm['name'],free))
        while bm['n_passengers'] < bm['capacity']:
            #waiting = [idx for idx, passenger in enumerate(passengers[current_node]) if passenger['bm'] == event.bm]
            waiting = [idx for idx, passenger in enumerate(passengers[current_node]) if passenger['station'] in bm['itinerary']]
            print(waiting)
            if not waiting:
                break
            passenger = passengers[current_node].pop(waiting[0])
            print(passenger)
            bm['n_passengers'] += 1
            hq.heappush(eventq, Event(event.time + passenger['exit_time'],'exit', event.bm))
            print(event.time+passenger['exit_time'])
        hq.heappush(eventq, Event(event.time+150, 'load', event.bm))
        bm['stop'] += 1
        # Generate events for exiting passengers
    elif event.action == 'exit':
        print('Fahrzeug {}, Passagiere exit'.format(bm['name']))
        print('Es sind {}'.format(bm['n_passengers']))
        bm['n_passengers'] -= 1
    else:
        raise Exception('Unknown event action')

Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[0, 1, 2, 3, 4, 5, 6, 7]
{'station': 1, 'exit_time': 60}
60
[0, 1, 2, 3, 4, 5, 6]
{'station': 1, 'exit_time': 60}
60
[0, 1, 2, 3, 4, 5]
{'station': 1, 'exit_time': 60}
60
Fahrzeug 1 ist am knoten 1
Nächterhalt des Fahrzeug 1 ist 2
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
{'station': 0, 'exit_time': 110}
110
[0, 1, 2, 3, 4, 5, 6, 7, 8]
{'station': 0, 'exit_time': 110}
110
[0, 1, 2, 3, 4, 5, 6, 7]
{'station': 0, 'exit_time': 110}
110
[0, 1, 2, 3, 4, 5, 6]
{'station': 0, 'exit_time': 110}
110
[0, 1, 2, 3, 4, 5]
{'station': 0, 'exit_time': 110}
110
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0, Passagiere exit
Es sind 3
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0, Passagiere exit
Es sind 2
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0, Passagiere exit
Es sind 1
Fahrzeug 1 ist am

Fahrzeug 1 ist am knoten 0
Nächterhalt des Fahrzeug 1 ist 1
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 1
Nächterhalt des Fahrzeug 1 ist 2
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 2
Nächterhalt des Fahrzeug 1 ist 0
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 0
Nächterhalt des Fahrzeug 1 ist 1
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 1
Nächterhalt des Fahrzeug 1 ist 2
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhal

[]
Fahrzeug 1 ist am knoten 0
Nächterhalt des Fahrzeug 1 ist 1
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 1
Nächterhalt des Fahrzeug 1 ist 2
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 2
Nächterhalt des Fahrzeug 1 ist 0
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 0
Nächterhalt des Fahrzeug 1 ist 1
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 1
Nächterhalt des Fahrzeug 1 ist 2
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächter

[]
Fahrzeug 1 ist am knoten 0
Nächterhalt des Fahrzeug 1 ist 1
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 1
Nächterhalt des Fahrzeug 1 ist 2
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 2
Nächterhalt des Fahrzeug 1 ist 0
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 0
Nächterhalt des Fahrzeug 1 ist 1
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 1
Nächterhalt des Fahrzeug 1 ist 2
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächter

Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 0
Nächterhalt des Fahrzeug 1 ist 1
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 1
Nächterhalt des Fahrzeug 1 ist 2
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 2
Nächterhalt des Fahrzeug 1 ist 0
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 0
Nächterhalt des Fahrzeug 1 ist 1
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 1
Nächterhalt des Fahrzeug 1 ist 2
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrze

...

[]
Fahrzeug 1 ist am knoten 0
Nächterhalt des Fahrzeug 1 ist 1
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 1
Nächterhalt des Fahrzeug 1 ist 2
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 2
Nächterhalt des Fahrzeug 1 ist 0
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 0
Nächterhalt des Fahrzeug 1 ist 1
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 1
Nächterhalt des Fahrzeug 1 ist 2
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächter

[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 2
Nächterhalt des Fahrzeug 1 ist 0
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 0
Nächterhalt des Fahrzeug 1 ist 1
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 1
Nächterhalt des Fahrzeug 1 ist 2
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 2
Nächterhalt des Fahrzeug 1 ist 0
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 0
Nächter

Fahrzeug 1 ist am knoten 1
Nächterhalt des Fahrzeug 1 ist 2
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 2
Nächterhalt des Fahrzeug 1 ist 0
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 0
Nächterhalt des Fahrzeug 1 ist 1
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 1
Nächterhalt des Fahrzeug 1 ist 2
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 2
Nächterhalt des Fahrzeug 1 ist 0
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhal

Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 2
Nächterhalt des Fahrzeug 1 ist 0
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 0
Nächterhalt des Fahrzeug 1 ist 1
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 1
Nächterhalt des Fahrzeug 1 ist 2
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 1
Nächterhalt des Fahrzeug 0 ist 0
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 2
Nächterhalt des Fahrzeug 1 ist 0
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrzeug 0 ist am knoten 0
Nächterhalt des Fahrzeug 0 ist 1
Fahrzeug 0 load
Fahrzeug 0 hat 3 Pläze frei
[]
Fahrzeug 1 ist am knoten 0
Nächterhalt des Fahrzeug 1 ist 1
Fahrzeug 1 load
Fahrzeug 1 hat 5 Pläze frei
[]
Fahrze

KeyboardInterrupt: 

In [ ]:
bms = [{'n_passengers': 0, 'capacity': 3, 'stop': 0, 'itinerary': [0, 1], 'name': 0},
       {'n_passengers': 0, 'capacity': 3, 'stop': 1, 'itinerary': [0, 1], 'name': 1}]
pint(evne)

In [ ]:
passengers = [[Passenger(0,120)]*5, [Passenger(0,120)]*2]
bms = [BM(0,3,0,[0,1])]
event0 = Event(0,'load',0) 
eventq = []
hq.heappush(eventq, event0)


while eventq:
    event = hq.heappop(eventq)
    print(event)
    bahn = bms[event.bm]
    curent_node = bahn.itinerary [bahn.stop % len(bahn.itinerary)]
    next_node =  bahn.itinerary [(bahn.stop +1)  % len(bahn.itinerary)]
    print(curent_node)
    print(next_node)
    if event.action == 'load':
        #free = bahn.capacity - bahn.passenger
        while bahn.passenger < bahn.capacity:
            waiting = [idx for idx, passenger in enumerate(passengers[curent_node]) if passenger.bm == event.bm]
            print(waiting)
            if not waiting:
                break
            passenger = passengers[curent_node].pop(waiting[0])
            print(passenger)
            bahn.passenger += 1
            hq.heappush(eventq, Event(event.time + passenger.exit,'exit', event.bm))
            print(eventq)
        hq.heappush(eventq, Event(event.time+150, 'load', event.bm))
        print(eventq)
        bahn.stop = next_node
    elif event.action == 'exit':
        bahn.passenger -= 1
    else:
        raise Exception('Unknown event action')